# Fluent Python
> Notes By zzf 20190527

> Reference: ***Fluent Python*** *By Luciano Ramalho*

## Chapter 1  Python 数据模型
> Python 最好的品质之一是一致性。当你使用 Python 工作一会儿后，就会开始理解 Python 语言，并能正确猜测出对你来说全新的语言特征。
然而，如果你带着来自其他面向对象语言的经验进入 Python 的世界，会对 len(colleciton) 而不是 collection.len() 写法觉得不适。当你进一步理解这种不适 感背后的原因之后，会发现这个原因，和它所代表的庞大的设计思想，是形成我们通常说 的“Python 风格”（Pythonic）的关键。这种设计思想完全体现在 Python 的数据模型上，而 数据模型所描述的 API，为使用最地道的语言特性来构建你自己的对象提供了工具。

### 特殊方法 / magic method / dunder method
```py
__init__, __len__, __getitem__
```

#### Exp 1.1 纸牌

In [1]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])
class FrenchDeck:    
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')    
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits 
                                        for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    def __getitem__(self, position):
        return self._cards[position]

##### Notes
```py
__len__(self) # 外部使用len()调用该类对象时，__len__起作用，进一层用len()调用list对象，由list对象的__len__起作用

__getitem__(self) # 外部使用[]调用该类对象时，__getitem__起作用，进一层用[]调用dict对象，由dict对象的__getitem__起作用
```

In [2]:
deck = FrenchDeck()

##### Notes
```py
# 由__len__实现
len(dect)

# 由__getitem__实现
deck[0]
deck[-1]
deck[12::13]

for card in deck: 
    print(card)

from random import choice
choice(deck) # Card(rank='3', suit='hearts')
choice(deck) # Card(rank='K', suit='spades') 
choice(deck) # Card(rank='2', suit='clubs')


```

In [3]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

##### Notes
```py
FrenchDeck.ranks # ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
FrenchDeck.ranks.index('A') # 12 -> return the index of the input element
rank_value * len(suit_values) + suit_values[card.suit] # get the 'score' of input card
```

In [4]:
for card in sorted(deck, key=spades_high):
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='5', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='10', suit='clubs')
Ca

##### Notes
```py
sorted(deck, key=spades_high) # sorted(iterable, key=function) # every element in iterable will apply the function
```

## Chapter 16 Coroutine

In [4]:
def simple_coroutine():
    print("-> Coroutine Started")
    x = yield
    print("-> Coroutine Received:", x)

In [6]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x0000022ED0766228>

In [7]:
my_coro = simple_coroutine()
for i in range(2):
    print(i)
    next(my_coro)

0
-> Coroutine Started
1
-> Coroutine Received: None


StopIteration: 

In [5]:
my_coro = simple_coroutine()
# 激活协程
next(my_coro)
# 上一步运行后协程运行到yield前一句，处于暂停状态，可以调用send()，恢复协程
my_coro.send("Testing String")
# 一直运行到下一个yield表达式之前或者终止
# 这里，控制权流动到协程定义体的末尾，导致生成器像往常一样抛出 StopIteration异常

-> Coroutine Started
-> Coroutine Received: Testing String


StopIteration: 

In [2]:
my_coro = simple_coroutine()
# 未激活协程无法调用send()
my_coro.send("Testing String")

NameError: name 'simple_coroutine' is not defined

> 最先调用`next(my_coro)`函数这一步通常称为“预激”（prime）协程（即，让协程向前执行到第一个`yield`表达式，准备好作为活跃的协程使用）

In [3]:
 from inspect import getgeneratorstate 
 
 def simple_coro2(a):
     print('-> Started: a =', a)
     b = yield a 
     print('-> Received: b =', b)
     c = yield a + b
     print('-> Received: c =', c) 

In [0]:
try:
    my_coro2 = simple_coro2(14)
    print(getgeneratorstate(my_coro2))
    # 先传出值(1st), 然后暂停，等待为 b 赋值
    print("yield a:", next(my_coro2))
    print(getgeneratorstate(my_coro2))
    # 传入(1st)，运行到第二个yield时传出(2nd)，然后暂停，等待为c赋值
    print("yield a+b:", my_coro2.send(28))
    # 传入(2nd)，运行，然后协程终止，导致生成器对象抛出 StopIteration异常
    print("last:", my_coro2.send(99))
except StopIteration as e:
    print(e)
    print(getgeneratorstate(my_coro2))

> 在赋值语句中，`=`右边的代码在赋值之前执行。因此，对于`b = yield a`这行代码来说，等到客户端代码再激活协程时才会设定`b`的值。这种行为要花点时间才能习惯，不过一定要理解，这样才能弄懂异步编程中`yield`的作用

In [4]:
from functools import wraps

def coroutine(func):
    """装饰器：向前执行到第一个`yield`表达式，预激`func`"""        
    @wraps(func)
    def primer(*args,**kwargs):
        gen = func(*args,**kwargs)
        next(gen)
        return gen
    
    return primer

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [5]:
coro_avg = averager()
print(getgeneratorstate(coro_avg))
for data in range(1, 11, 4):
    print(coro_avg.send(data))

GEN_SUSPENDED
1.0
3.0
5.0


> 使用 `ield from`句法（参见 16.7 节）调用协程时，会自动预激，因此与示例 16-5 中的`@coroutine`等装饰器不兼容。Python 3.4 标准库里的`asyncio.coroutine`装饰器（第 18 章介绍）不会预激协程，因此能兼容`yield from`句法

In [6]:
try:
    coro_avg.send("String")
except TypeError as e:
    print(e)
    print(getgeneratorstate(coro_avg))

coro_avg.send("String")

unsupported operand type(s) for +=: 'float' and 'str'
GEN_CLOSED


StopIteration: 

> 示例 16-7 暗示了终止协程的一种方式：发送某个哨符值，让协程退出。内置的 None 和 Ellipsis 等常量经常用作哨符值。Ellipsis 的优点是，数据流中不太常有这个值。我还见过有人把 StopIteration 类（类本身，而不是实例，也不抛出）作为哨符值；也就是说，是像这样使用的：my_coro.send(StopIteration)。 

In [7]:
from collections import namedtuple


data = {
    'girls;kg': [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m': [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

Result = namedtuple('Result', 'count average')

# the subgenerator
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

# the delegating generator
def grouper(results, key):
    count = 0
    while True:
        # 自动处理异常并提取返回值，处理异常后while loop中止
        results[key] = yield from averager()
        count += 1
        print("while loop", count, results, key)

# the client code, a.k.a. the caller
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)  # important!

    # print(results)  # uncomment to debug
    report(results)

# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(result.count, group, result.average, unit))

In [8]:
main(data)

while loop 1 {'girls;kg': Result(count=10, average=42.040000000000006)} girls;kg
while loop 1 {'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997)} girls;m
while loop 1 {'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224)} boys;kg
while loop 1 {'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)} boys;m
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


## Chapter 17  使用`future`处理并发

In [9]:
import os, time, sys, requests

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = 'downloads/'

In [10]:
class Flags:
    
    @staticmethod
    def save_flag(img, filename):
        path = os.path.join(DEST_DIR, filename)
        with open(path, 'wb') as fp:
            fp.write(img)
    
    @staticmethod
    def get_flag(cc):
        url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
        resp = requests.get(url)
        return resp.content
    
    @staticmethod
    def show(text):
        print(text, end=',')
        sys.stdout.flush()
    
    @classmethod
    def download_many(cls, cc_list):
        for cc in sorted(cc_list):
            image = cls.get_flag(cc)
            cls.show(cc)
            cls.save_flag(image, cc.lower() + '.gif')

        return len(cc_list)
    
    @classmethod
    def main(cls):
        t0 = time.time()
        count = cls.download_many(POP20_CC)
        elapsed = time.time() - t0
        msg = '\n{} flags downloaded in {:.2f}s'
        print(msg.format(count, elapsed))


In [11]:
Flags.main()

BD,BR,CD,CN,DE,EG,ET,FR,ID,IN,IR,JP,MX,NG,PH,PK,RU,TR,US,VN,
20 flags downloaded in 39.14s


In [12]:
from concurrent import futures


MAX_WORKERS = 20


class Flags_Threadpool(Flags):

    @classmethod
    def download_one(cls, cc):
        image = cls.get_flag(cc)
        cls.show(cc)
        cls.save_flag(image, cc.lower() + '.gif')
        return cc

    @classmethod
    def download_many(cls, cc_list):
        workers = min(MAX_WORKERS, len(cc_list))
        with futures.ThreadPoolExecutor(workers) as executor:
            res = executor.map(cls.download_one, sorted(cc_list))

        return len(list(res))

In [13]:
Flags_Threadpool.main()

BD,ID,RU,FRDE,,JP,IR,MX,CN,EG,CD,TR,PK,US,ET,BR,VN,NG,IN,PH,
20 flags downloaded in 6.00s


In [14]:
import asyncio
import aiohttp

class Flags_Asyncio(Flags):

    @staticmethod
    async def get_flag(session, cc):
        url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
        async with session.get(url) as resp:
            return await resp.read()

    @classmethod
    async def download_one(cls, session, cc):
        image = await cls.get_flag(session, cc)
        cls.show(cc)
        cls.save_flag(image, cc.lower() + '.gif')
        return cc

    @classmethod
    async def download_many(cls, cc_list):
        async with aiohttp.ClientSession() as session:
            res = await asyncio.gather(
                *[asyncio.create_task(cls.download_one(session, cc))
                    for cc in sorted(cc_list)])

        return len(res)

In [15]:
Flags_Asyncio.main()


<coroutine object Flags_Asyncio.download_many at 0x00000240334467C8> flags downloaded in 0.00s
C:\Users\Nature\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: coroutine 'Flags_Asyncio.download_many' was never awaited
  """Entry point for launching an IPython kernel.


In [1]:
from time import perf_counter, sleep
from itertools import cycle

def spin(during=10):
    start = perf_counter()
    for char in cycle("|/-\\"):
        print(char, flush=True, end="\r")
        sleep(0.07)
        if perf_counter() - start >= during:
            break

In [3]:
spin(2)

|/-\|/-\|/-\|/-\|/-\|/-\|/-\

## Chapter 18 Concurrency with `asyncio`

In [6]:
import types

help(types.coroutine)

Help on function coroutine in module types:

coroutine(func)
    Convert regular generator function to a coroutine.

